In [31]:
import cv2
import numpy as np

# Detector de Harris

In [17]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    # extract keypoints
    points = np.argwhere(dst > threshold * dst.max())
    keypoints = [cv2.KeyPoint(float(x[1]), float(x[0]), 13) for x in points]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image 


In [18]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
kp, kp_image = detectHarrisKeypoints(image)
cv2.imshow('haris_corner', kp_image) 
cv2.waitKey() 

-1

# Detector SIFT

In [19]:
def detectSIFTKeypoints(image):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create() 
    kp, des = sift.detectAndCompute(gray_image, None) 
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=0) 

    return kp, kp_image 

In [20]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
kp, kp_image = detectSIFTKeypoints(image)
cv2.imshow('SIFT', kp_image) 
cv2.waitKey() 

-1

# ORB Detection

In [21]:
def detectORBKeypoits(image):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    orb = cv2.ORB_create() 
    kp, des = orb.detectAndCompute(gray_image, None)    

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 

In [22]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
kp, kp_image = detectORBKeypoits(image)
cv2.imshow('ORB', kp_image) 
cv2.waitKey() 

-1

# Comparing Keypoints

In [23]:
import numpy as np

def computeDistacesKeypoints(kp1, kp2, threshold=1):
    matches = []
    for k in kp1:
        pt = k.pt
        pts = cv2.KeyPoint_convert(kp2)
        dists = np.linalg.norm(pt - pts, axis=1)
        tuple_dist = [(pt, dist) for pt, dist in zip(list(pts), dists)]
        tuple_dist.sort(key=lambda x: x[1])

        if tuple_dist[0][1] <= threshold:
            matches.append(tuple_dist[0][0])
    
    return matches


In [24]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
h_kp, harris_image = detectHarrisKeypoints(image)
s_kp, sift_image = detectSIFTKeypoints(image)
o_kp, orb_image = detectORBKeypoits(image)


In [25]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=1)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=1)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=1)

In [26]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD ORB Keypoints:   {len(o_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/ORB:   {len(matches_ho)}')
print(f'Matches Keypoints SIFT/ORB:   {len(matches_so)}')

----- Resultados -----
QTD Harris Keypoints:   9288
QTD SIFT Keypoints:   1518
QTD ORB Keypoints:   500
----- Matches -----
Matches Keypoints Harris/SIFT:   645
Matches Keypoints Harris/ORB:   662
Matches Keypoints SIFT/ORB:   80


In [27]:
kp_matches_hs = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_hs]
kp_matches_ho = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_ho]
kp_matches_so = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_so]

In [28]:
kp1_image = cv2.drawKeypoints(image, kp_matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, kp_matches_ho, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, kp_matches_so, None, color=(255, 0, 0), flags=0)

In [ ]:
cv2.imshow('ORB', kp_image)

# SIFT Descriptor

In [32]:
# Read the query image as query_img 
# and train image This query image 
# is what you need to find in train image 
# Save it in the same directory 
# with the name image.jpg
   
query_img = cv2.imread('../data/imgs/dsc07631.jpg') 
train_img = cv2.imread('../data/imgs/dsc07632.jpg') 
   
# Convert it to grayscale 
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY) 
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY) 
   
# Initialize the ORB detector algorithm 
orb = cv2.SIFT_create(nfeatures=50) 
   
# Now detect the keypoints and compute 
# the descriptors for the query image 
# and train image 
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None) 
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None) 
  
# Initialize the Matcher for matching 
# the keypoints and then match the 
# keypoints 
matcher = cv2.BFMatcher() 
matches = matcher.match(queryDescriptors,trainDescriptors) 
   
# draw the matches to the final image 
# containing both the images the drawMatches() 
# function takes both images and keypoints 
# and outputs the matched query image with 
# its train image 
final_img = cv2.drawMatches(query_img, queryKeypoints,  
train_img, trainKeypoints, matches, None) 
   
final_img = cv2.resize(final_img, (1000,650)) 
  
# Show the final image 
cv2.imshow("Matches", final_img) 
cv2.waitKey()

In [30]:
orb.getNOctaveLayers()

3

# ORB Descriptor